In [1]:
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from IPython.display import display

In [2]:
gis = GIS()
map1 = gis.map()
map1

MapView(layout=Layout(height='400px', width='100%'))

In [3]:
# set the map's extent by geocoding the location
diegogarcia = geocode('Diego Garcia')[0]
map1.extent = diegogarcia['extent']

In [4]:
# geocode location of La Reunion island
lareunion = geocode('La Reunion')[0]

In [5]:
# Annotate the map by plotting Diego Garcia, and two other search locations
map1.draw(lareunion['location'], {"title": "Reunion Island", "content": "Debris found"})
map1.draw(diegogarcia['location'], {"title": "Diego Garcia", "content": "Naval Support Facility Diego Garcia"})
map1.draw([-43.5, 90.5], {"title": "Search Location", "content": "Predicted crash location"})

# Render a feature layer representing the search area
# Source: http://www.amsa.gov.au/media/incidents/images/DIGO_00718_01_14.jpg
map1.add_layer({"type": "FeatureLayer",
                "url": "http://services.arcgis.com/WQ9KVmV6xGGMnCiQ/arcgis/rest/services/MH370Search/FeatureServer/1"})

In [6]:
mh370items = gis.content.search("MH370", "feature service", max_items=6)
for item in mh370items:
    display(item)

<Item title:"Possible_MH370_Debris_Recovered" type:Feature Layer Collection owner:geoscienceaustralia>

<Item title:"The_7th_Arc" type:Feature Layer Collection owner:geoscienceaustralia>

<Item title:"MH370 Phase 2 contacts" type:Feature Layer Collection owner:geoscienceaustralia>

<Item title:"Search_And_Rescue_Search_Area" type:Feature Layer Collection owner:geoscienceaustralia>

<Item title:"MH370_Flight_Path" type:Feature Layer Collection owner:bflanagan_bureau>

<Item title:"RADAR_Polylines" type:Feature Layer Collection owner:geoscienceaustralia>

In [7]:
map1.add_layer(mh370items[0])
map1.add_layer(mh370items[4])
map1.add_layer(mh370items[5])

In [8]:
map1.zoom = 6

# Using geoprocessing tools

In [9]:
toolbox_item = gis.content.search("Ocean Currents", item_type="geoprocessing toolbox", max_items=1)[0]
toolbox_item

<Item title:"Farthest On Circle" type:Geoprocessing Toolbox owner:johnfry10>

In [ ]:
from arcgis.geoprocessing import import_toolbox
ocean_currents = import_toolbox(toolbox_item)

In [ ]:
help(ocean_currents.message_in_a_bottle)

In [ ]:
from arcgis.features import FeatureSet, Feature

def do_analysis(m, g):
    print("Computing the path that debris would take...")
    
    # Parameter 'g' contains the co-ordinates of the clicked location
    
    # Construct a FeatureSet object from the clicked location
    my_feature_set = FeatureSet([Feature(g)])
    
    # Pass the input location as a FeatureSet
    ret = ocean_currents.message_in_a_bottle(my_feature_set, 150)
    
    # Render the resulting FeatureSet on the map using 'draw()' method
    map1.draw(ret)

# et the callback function that performs analysis.
# The 'do_analysis' function is called whenever user clicks on the map.
map1.on_click(do_analysis)